In [ ]:
%run -n main.py

In [ ]:
lines = read_lines(DOTENV_PATH)
env = dict(parse_dotenv(lines))
openai.api_key = env['OPENAI_TOKEN']

# tasks

In [56]:
table = pd.read_csv('tasks/ru_user_oriented.csv')
table = table[~table.done.isnull()]
table = table.where(pd.notnull(table), None)
table = table[['id', 'instruction', 'input']]
ru_user_oriented_items = table.to_dict(orient='records')

# evals

## gpt-3.5-turbo

In [59]:
lines = read_lines('evals/openai_turbo.jsonl')
eval_items = list(parse_jsonl(lines))
cache_ids = {_['id'] for _ in eval_items}

In [60]:
items = [
    _ for _ in ru_user_oriented_items
    if _['id'] not in cache_ids
]
for item in log_progress(items):
    prompt = user_oriented_openai_prompt(item)
    output = openai_chat_complete(prompt, model='gpt-3.5-turbo')
    item = {
        'id': item['id'],
        'prompt': prompt,
        'output': output
    }
    eval_items.append(item)

100%|██████████| 47/47 [07:17<00:00,  9.30s/it]


In [61]:
%run -n main.py
lines = format_jsonl(eval_items)
write_lines('evals/openai_turbo.jsonl', lines)

## text-davinci-003

In [62]:
lines = read_lines('evals/openai_davinci_003.jsonl')
eval_items = list(parse_jsonl(lines))
cache_ids = {_['id'] for _ in eval_items}

In [63]:
items = [
    _ for _ in ru_user_oriented_items
    if _['id'] not in cache_ids
]
for item in log_progress(items):
    prompt = user_oriented_openai_prompt(item)
    output = openai_complete(prompt, model='text-davinci-003')
    item = {
        'id': item['id'],
        'prompt': prompt,
        'output': output
    }
    eval_items.append(item)

0it [00:00, ?it/s]


In [ ]:
%run -n main.py
lines = format_jsonl(eval_items)
write_lines('evals/openai_davinci_003.jsonl', lines)

## text-davinci-002

In [64]:
lines = read_lines('evals/openai_davinci_002.jsonl')
eval_items = list(parse_jsonl(lines))
cache_ids = {_['id'] for _ in eval_items}

In [65]:
items = [
    _ for _ in ru_user_oriented_items
    if _['id'] not in cache_ids
]
for item in log_progress(items):
    prompt = user_oriented_openai_prompt(item)
    output = openai_complete(prompt, model='text-davinci-002')
    item = {
        'id': item['id'],
        'prompt': prompt,
        'output': output
    }
    eval_items.append(item)

0it [00:00, ?it/s]


In [ ]:
%run -n main.py
lines = format_jsonl(eval_items)
write_lines('evals/openai_davinci_002.jsonl', lines)

## davinci-001

In [69]:
lines = read_lines('evals/openai_davinci_001.jsonl')
eval_items = list(parse_jsonl(lines))
cache_ids = {_['id'] for _ in eval_items}

In [70]:
items = [
    _ for _ in ru_user_oriented_items
    if _['id'] not in cache_ids
]
for item in log_progress(items):
    prompt = user_oriented_openai_prompt(item)
    output = openai_complete(prompt, model='davinci')
    item = {
        'id': item['id'],
        'prompt': prompt,
        'output': output
    }
    eval_items.append(item)

  0%|          | 0/1 [00:00<?, ?it/s]


InvalidRequestError: This model's maximum context length is 2049 tokens, however you requested 2302 tokens (1278 in your prompt; 1024 for the completion). Please reduce your prompt; or completion length.

In [68]:
%run -n main.py
lines = format_jsonl(eval_items)
write_lines('evals/openai_davinci_001.jsonl', lines)